In [11]:
#Total combinations are 70 since 8c4 = 8!/(4!*(8-4)!)=70
from math import comb
print(comb(8,4))


70


In [ ]:
from math import log2

#Total information needed is:
#H(x) = -Σp(x)log2(p(x))
information = - (1/70)*70*log2(1/70)
print(f"\nTotal information needed: {information} bits")


Total information needed: 6.129283016944966 bits


This is not 8 bits of information, since you know that exactly 4 positions are taken. 


In [17]:
from itertools import combinations

# Generate all possible 8-digit binary numbers with sum of 4
binary_numbers = []

# Get all combinations of 4 ones in 8 positions
positions = list(combinations(range(8), 4))

# Convert each combination to a binary number
for pos in positions:
    # Create list of 8 zeros 
    binary = [0] * 8
    
    # Place 1s at the selected positions
    for p in pos:
        binary[p] = 1
        
    # Convert to integer by joining and parsing binary string
    binary_number = int(''.join(map(str, binary)), 2)
    binary_numbers.append(binary_number)


# Print results
for num in binary_numbers:
    print(bin(num))

print(f"\nTotal combinations: {len(binary_numbers)}")


0b11110000
0b11101000
0b11100100
0b11100010
0b11100001
0b11011000
0b11010100
0b11010010
0b11010001
0b11001100
0b11001010
0b11001001
0b11000110
0b11000101
0b11000011
0b10111000
0b10110100
0b10110010
0b10110001
0b10101100
0b10101010
0b10101001
0b10100110
0b10100101
0b10100011
0b10011100
0b10011010
0b10011001
0b10010110
0b10010101
0b10010011
0b10001110
0b10001101
0b10001011
0b10000111
0b1111000
0b1110100
0b1110010
0b1110001
0b1101100
0b1101010
0b1101001
0b1100110
0b1100101
0b1100011
0b1011100
0b1011010
0b1011001
0b1010110
0b1010101
0b1010011
0b1001110
0b1001101
0b1001011
0b1000111
0b111100
0b111010
0b111001
0b110110
0b110101
0b110011
0b101110
0b101101
0b101011
0b100111
0b11110
0b11101
0b11011
0b10111
0b1111

Total combinations: 70


In [ ]:
def get_information_gain(d):
    # size of d
    n = sum(d.values())
    ans = 0
    for key in d:
        ans += (d[key]/n)*log2(n/d[key])
    return ans

def guess(guessed_num, possibilities):
    d = {}
    ans = {}
    for number in possibilities:
        num = number & guessed_num 
        val = bin(num).count('1')
        
        # Combine all values below 3 into 0
        if val < 3:
            val = 0
            
        if val in d:
            d[val] += 1
        else:
            d[val] = 1
        if val in ans:
            ans[val].append(number) 
        else:
            ans[val] = [number]

    return d, ans
# def best_option(remaining_possibilities, all_possibilities):
#     if len(remaining_possibilities) == 1:
#         return remaining_possibilities[0]
#     best = 0
#     best_guess = 0
#     for guessed_num in all_possibilities:
#         sum_dict, possibilities_dict = guess(guessed_num, remaining_possibilities)
#         information_gain = get_information_gain(sum_dict)
#         if information_gain > best:
#             best_guess = guessed_num
#             best = information_gain
#     return best_guess

def best_option(remaining_possibilities, all_possibilities):
    if len(remaining_possibilities) == 1:
        return remaining_possibilities[0]
    best = 0
    best_guess = 0
    for guessed_num in all_possibilities:            
        sum_dict, possibilities_dict = guess(guessed_num, remaining_possibilities)
        information_gain = get_information_gain(sum_dict)
        if information_gain > best:
            best_guess = guessed_num
            best = information_gain
        
    return best_guess

def generate_decision_tree(guessed_num, possibilities, all_possibilities, decision_tree=None):
    if decision_tree is None:
        decision_tree = {}
    
    if len(possibilities) == 1:
        decision_tree["Final"] = possibilities[0]  # Store the final answer
        return decision_tree
    
    sum_dict, possibilities_dict = guess(guessed_num, possibilities)
    
    # For each possible outcome of our guess
    for key in possibilities_dict:
        next_guess = best_option(possibilities_dict[key], all_possibilities)
        # Create a new branch in our decision tree
        if key not in decision_tree:
            decision_tree[key] = {'Guess': format(next_guess, '08b'), "d": sum_dict, 'subtree': {}}
            
        # Recursively build the rest of this branch
        decision_tree[key]['subtree'] = generate_decision_tree(
            next_guess,
            possibilities_dict[key],
            all_possibilities,
            {}
        )
    
    return decision_tree
def pretty_print_tree(tree, indent=0, prefix=''):
    if not isinstance(tree, dict):
        print(f"{' ' * indent}│── Answer: {format(tree, '08b')}")
        return

    items = sorted(tree.items(), key=lambda x: not isinstance(x[1], dict))
    
    first_value = items[0][1]
    if isinstance(first_value, dict) and 'Guess' in first_value:
        print(f"{' ' * indent}├── Guess: {first_value['Guess']}")
        if 'd' in first_value:
            dist_str = ', '.join(f"{k}:{v}" for k, v in first_value['d'].items())
            print(f"{' ' * indent}│   Distribution: [{dist_str}]")
    
    for key, value in items:
        if isinstance(value, dict):
            label = {4: "Correct", 3: "One off", 0: "No match"}.get(key, f"Response {key}")
            print(f"{' ' * indent}│")
            print(f"{' ' * indent}├── {label}:")
            pretty_print_tree(value['subtree'], indent + 4)
        else:
            print(f"{' ' * indent}│── Answer: {format(value, '08b')}")

def analyze_path(tree, target, path=None):
    if path is None:
        path = []
        
    if not isinstance(tree, dict):
        return path if format(tree, '08b') == target else None
        
    for key, value in tree.items():
        if isinstance(value, dict) and 'Guess' in value:
            guess = value['Guess']
            path.append(guess)
            result = analyze_path(value['subtree'], target, path)
            if result:
                return result
            path.pop()
    
    return None

def interactive_game(tree):
    current = tree
    steps = 0
    
    while isinstance(current, dict):
        first_dict = next(v for v in current.values() if isinstance(v, dict))
        guess = first_dict['Guess']
        steps += 1
        
        print(f"\nStep {steps}: Try {guess}")
        response = input("Enter result (4=correct, 3=one-off, 0=no match): ").strip()
        
        try:
            response = int(response)
            if response in current:
                current = current[response]['subtree']
            else:
                print("Invalid response. Try again.")
                steps -= 1
        except ValueError:
            print("Please enter a valid number")
            steps -= 1
    
    print(f"\nFound answer: {format(current, '08b')} in {steps} steps!")

def get_tree_depths(tree, current_depth=0):
    depths = []
    
    # Base case: if it's not a dictionary or it's a leaf node
    if not isinstance(tree, dict):
        return [current_depth]
    
    # For each branch in the tree
    for key, value in tree.items():
        if isinstance(value, dict):
            # If it has a subtree, recursively get depths
            sub_depths = get_tree_depths(value['subtree'], current_depth + 1)
            depths.extend(sub_depths)
        else:
            # If it's a leaf node
            depths.append(current_depth)
            
    return depths

def analyze_tree_depth(tree):
    depths = get_tree_depths(tree)
    
    max_guesses = max(depths)
    avg_guesses = sum(depths) / len(depths)
    num_leaves = len(depths)
    
    print(f"Maximum number of guesses needed: {max_guesses}")
    print(f"Average number of guesses needed: {avg_guesses:.2f}")
    print(f"Number of possible outcomes: {num_leaves}")
    
    # Print distribution of guess counts
    guess_distribution = {}
    for depth in depths:
        guess_distribution[depth] = guess_distribution.get(depth, 0) + 1
    
    print("\nDistribution of number of guesses:")
    for guesses, count in sorted(guess_distribution.items()):
        print(f"{guesses} guesses: {count} outcomes ({(count/num_leaves)*100:.1f}%)")

# Analyze the tree

decision_tree = generate_decision_tree(0b11110000, binary_numbers, binary_numbers)



In [149]:
pretty_print_tree(decision_tree)
analyze_tree_depth(decision_tree)


├── Guess: 11110000
│   Distribution: [4:1, 3:16, 0:53]
│
├── Correct:
    │── Answer: 11110000
│
├── One off:
    ├── Guess: 11100010
    │   Distribution: [3:4, 0:12]
    │
    ├── One off:
        ├── Guess: 11011000
        │   Distribution: [3:2, 0:2]
        │
        ├── One off:
            ├── Guess: 11101000
            │   Distribution: [3:1, 0:1]
            │
            ├── One off:
                │── Answer: 11101000
            │
            ├── No match:
                │── Answer: 11100100
        │
        ├── No match:
            ├── Guess: 11011000
            │   Distribution: [3:1, 0:1]
            │
            ├── One off:
                │── Answer: 11011000
            │
            ├── No match:
                │── Answer: 11010100
    │
    ├── No match:
        ├── Guess: 11100100
        │   Distribution: [3:4, 0:8]
        │
        ├── One off:
            ├── Guess: 11010010
            │   Distribution: [3:2, 0:2]
            │
            ├── One o

In [139]:
def get_tree_depths(tree, current_depth=0):
    depths = []
    
    # Base case: if it's not a dictionary or it's a leaf node
    if not isinstance(tree, dict):
        return [current_depth]
    
    # For each branch in the tree
    for key, value in tree.items():
        if isinstance(value, dict):
            # If it has a subtree, recursively get depths
            sub_depths = get_tree_depths(value['subtree'], current_depth + 1)
            depths.extend(sub_depths)
        else:
            # If it's a leaf node
            depths.append(current_depth)
            
    return depths

def analyze_tree_depth(tree):
    depths = get_tree_depths(tree)
    
    max_guesses = max(depths)
    avg_guesses = sum(depths) / len(depths)
    num_leaves = len(depths)
    
    print(f"Maximum number of guesses needed: {max_guesses}")
    print(f"Average number of guesses needed: {avg_guesses:.2f}")
    print(f"Number of possible outcomes: {num_leaves}")
    
    # Print distribution of guess counts
    guess_distribution = {}
    for depth in depths:
        guess_distribution[depth] = guess_distribution.get(depth, 0) + 1
    
    print("\nDistribution of number of guesses:")
    for guesses, count in sorted(guess_distribution.items()):
        print(f"{guesses} guesses: {count} outcomes ({(count/num_leaves)*100:.1f}%)")

# Analyze the tree
analyze_tree_depth(decision_tree)


Maximum number of guesses needed: 10
Average number of guesses needed: 6.97
Number of possible outcomes: 70

Distribution of number of guesses:
1 guesses: 1 outcomes (1.4%)
4 guesses: 4 outcomes (5.7%)
5 guesses: 11 outcomes (15.7%)
6 guesses: 18 outcomes (25.7%)
7 guesses: 8 outcomes (11.4%)
8 guesses: 8 outcomes (11.4%)
9 guesses: 12 outcomes (17.1%)
10 guesses: 8 outcomes (11.4%)


In [145]:
def pretty_print_tree(tree, indent=0, prefix=''):
    if not isinstance(tree, dict):
        print(f"{' ' * indent}│── Answer: {format(tree, '08b')}")
        return

    items = sorted(tree.items(), key=lambda x: not isinstance(x[1], dict))
    
    first_value = items[0][1]
    if isinstance(first_value, dict) and 'Guess' in first_value:
        print(f"{' ' * indent}├── Guess: {first_value['Guess']}")
        if 'd' in first_value:
            dist_str = ', '.join(f"{k}:{v}" for k, v in first_value['d'].items())
            print(f"{' ' * indent}│   Distribution: [{dist_str}]")
    
    for key, value in items:
        if isinstance(value, dict):
            label = {4: "Correct", 3: "One off", 0: "No match"}.get(key, f"Response {key}")
            print(f"{' ' * indent}│")
            print(f"{' ' * indent}├── {label}:")
            pretty_print_tree(value['subtree'], indent + 4)
        else:
            print(f"{' ' * indent}│── Answer: {format(value, '08b')}")

def analyze_path(tree, target, path=None):
    if path is None:
        path = []
        
    if not isinstance(tree, dict):
        return path if format(tree, '08b') == target else None
        
    for key, value in tree.items():
        if isinstance(value, dict) and 'Guess' in value:
            guess = value['Guess']
            path.append(guess)
            result = analyze_path(value['subtree'], target, path)
            if result:
                return result
            path.pop()
    
    return None

def interactive_game(tree):
    current = tree
    steps = 0
    
    while isinstance(current, dict):
        first_dict = next(v for v in current.values() if isinstance(v, dict))
        guess = first_dict['Guess']
        steps += 1
        
        print(f"\nStep {steps}: Try {guess}")
        response = input("Enter result (4=correct, 3=one-off, 0=no match): ").strip()
        
        try:
            response = int(response)
            if response in current:
                current = current[response]['subtree']
            else:
                print("Invalid response. Try again.")
                steps -= 1
        except ValueError:
            print("Please enter a valid number")
            steps -= 1
    
    print(f"\nFound answer: {format(current, '08b')} in {steps} steps!")
pretty_print_tree(decision_tree)

├── Guess: 11110000
│   Distribution: [4:1, 3:16, 0:53]
│
├── Correct:
    │── Answer: 11110000
│
├── One off:
    ├── Guess: 11100010
    │   Distribution: [3:4, 0:12]
    │
    ├── One off:
        ├── Guess: 11011000
        │   Distribution: [3:2, 0:2]
        │
        ├── One off:
            ├── Guess: 11101000
            │   Distribution: [3:1, 0:1]
            │
            ├── One off:
                │── Answer: 11101000
            │
            ├── No match:
                │── Answer: 11100100
        │
        ├── No match:
            ├── Guess: 11011000
            │   Distribution: [3:1, 0:1]
            │
            ├── One off:
                │── Answer: 11011000
            │
            ├── No match:
                │── Answer: 11010100
    │
    ├── No match:
        ├── Guess: 11100100
        │   Distribution: [3:4, 0:8]
        │
        ├── One off:
            ├── Guess: 11010010
            │   Distribution: [3:2, 0:2]
            │
            ├── One o